In [1]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging, shutil, papermill, re
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
from tqdm.notebook import tqdm
from typing import Literal


In [2]:
folder = Path("/home/julienb/Documents/Data/Luisa/")
scripts =  ["plot_heatmap.ipynb"] #["task_graph.ipynb", "events.ipynb", "plot_rt_mt.ipynb", "trial_data.ipynb", "plot_heatmap.ipynb"]
selection: Literal["base",  "subject", "session", "existing"] = ["session"]
copy: Literal["force", "notexisting"]= "force"
exec: Literal["force", "notexisting", "notsucess", "erase"]= "force"

In [3]:
dfs = []
for i, script in enumerate(scripts):
    for sel in selection:
        df = pd.DataFrame()
        if sel=="base":
            df["base_path"] = list(folder.glob("analysis"))
        elif sel=="subject":
            df["base_path"] = list(folder.glob("*/analysis"))
        elif sel=="session":
            df["base_path"] = list(folder.glob("**/Session--*/analysis"))
        elif sel=="existing":
            df["base_path"] = [f.parent for f in folder.glob(f"**/{script}")]
        df.insert(0, "quick_path", df["base_path"].apply(lambda p: str(Path(*p.parts[-3:-1]))))
        dfs.append(df.assign(script=Path(script), script_index=i, script_path=df.apply(lambda row: (row["base_path"]/script), axis=1)))
tasks = pd.concat(dfs).sort_values(["base_path", "script_index"])
tasks

,quick_path,base_path,script,script_index,script_path
119,opto_ALLTRIALS/Session--Rat101_20240911,/home/julienb/Documents/Data/Luisa/Rat101/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
118,opto_ALLTRIALS/Session--Rat101_20240912,/home/julienb/Documents/Data/Luisa/Rat101/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
112,opto_CueRT/Session--Rat101_20240828,/home/julienb/Documents/Data/Luisa/Rat101/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
115,opto_CueRT/Session--Rat101_20240829,/home/julienb/Documents/Data/Luisa/Rat101/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
109,opto_CueRT/Session--Rat101_20240830,/home/julienb/Documents/Data/Luisa/Rat101/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
...,...,...,...,...,...
16,opto_MT/Session--Rat246_20240801,/home/julienb/Documents/Data/Luisa/Rat246/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...
22,opto_MT/Session--Rat246_20240802,/home/julienb/Documents/Data/Luisa/Rat246/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...
20,opto_MT/Session--Rat246_20240805,/home/julienb/Documents/Data/Luisa/Rat246/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...
21,opto_MT/Session--Rat246_20240806,/home/julienb/Documents/Data/Luisa/Rat246/opto...,plot_heatmap.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...


In [4]:

if copy=="force": tasks["copy"] = True 
elif copy=="notexisting": tasks["copy"] = tasks["script_path"].apply(lambda p: not p.exists())
else: raise Exception(f"Unknown copy param {copy}")

if exec=="force": 
    tasks["exec"] = True
    tasks["erase"] = True
elif exec=="notexisting": 
    tasks["exec"] = tasks["script_path"].apply(lambda p: not p.with_suffix("").exists()) 
    tasks["erase"] = False
elif exec=="notsucess": 
    tasks["exec"] = tasks["script_path"].apply(lambda p: not (p.with_suffix("")/"notebook.html").exists())
    tasks["erase"] = False
elif exec=="erase":
    tasks["exec"] = False
    tasks["erase"] = True
else: raise Exception(f"Unknown exec param {exec}")

display(tasks.groupby(["copy", "exec", "script"])["base_path"].count().rename("count").to_frame().reset_index())
print(tasks.sort_values(["copy", "exec"]).to_string())


,copy,exec,script,count
0,True,True,plot_heatmap.ipynb,183


                                  quick_path                                                                                       base_path              script  script_index                                                                                                        script_path  copy  exec  erase
119  opto_ALLTRIALS/Session--Rat101_20240911      /home/julienb/Documents/Data/Luisa/Rat101/opto_ALLTRIALS/Session--Rat101_20240911/analysis  plot_heatmap.ipynb             0      /home/julienb/Documents/Data/Luisa/Rat101/opto_ALLTRIALS/Session--Rat101_20240911/analysis/plot_heatmap.ipynb  True  True   True
118  opto_ALLTRIALS/Session--Rat101_20240912      /home/julienb/Documents/Data/Luisa/Rat101/opto_ALLTRIALS/Session--Rat101_20240912/analysis  plot_heatmap.ipynb             0      /home/julienb/Documents/Data/Luisa/Rat101/opto_ALLTRIALS/Session--Rat101_20240912/analysis/plot_heatmap.ipynb  True  True   True
112      opto_CueRT/Session--Rat101_20240828          /home/julienb/Docum

In [5]:
copy_df = tasks.loc[tasks["copy"]]
progress = tqdm(copy_df[["base_path", "script"]].iterrows(), total=len(copy_df.index))
for _, (p,s) in progress:
    shutil.copy(s, p/s)

  0%|          | 0/183 [00:00<?, ?it/s]

In [6]:
erase_df = tasks.loc[tasks["erase"]]
progress = tqdm(erase_df[["base_path", "script"]].iterrows(), total=len(erase_df.index))
for _, (p,s) in progress:
    if (p/s.stem).exists():
        shutil.rmtree(p/s.stem)

  0%|          | 0/183 [00:00<?, ?it/s]

In [7]:
%env PYDEVD_DISABLE_FILE_VALIDATION=1
exec_df = tasks.loc[tasks["exec"]]
base_path_grp = exec_df.groupby("base_path")
progress = tqdm(base_path_grp, total=len(base_path_grp))
errors=[]

for p, grp in progress:
    quick_path = grp["quick_path"].iat[0]
    progress.set_description_str(quick_path)
    grp = grp.sort_values("script_index")
    skip=False
    for script in grp["script"]:
        if not skip:
            progress.set_postfix_str(f"n_prev_err={len(errors)}, Executing {script}")
            (p/script.stem).mkdir(exist_ok=True, parents=True)
            try:
                papermill.execute_notebook(p/script, p/script.stem/"notebook.ipynb", cwd=p, progress_bar=False,  log_ouput=True)
            except Exception as e:
                errors.append((quick_path, p, script, e))
                e.add_note(f"During execution of {script} in {p}")
                display(print(f"During execution of {script} in {p}", e))
                skip=True
                # raise
                
if len(errors) > 0:
    errors_df = pd.DataFrame(errors, columns=["quick_path", "base_path", "script", "exception"])
    errors_df["err_message"] = errors_df["exception"].apply(lambda e: re.findall(r'Exception:.*$', str(e)))
    display(errors_df[["quick_path", "script", "err_message"]])
    raise ExceptionGroup("Errors during execution", [e[-1] for e in errors])

env: PYDEVD_DISABLE_FILE_VALIDATION=1


  0%|          | 0/183 [00:00<?, ?it/s]

During execution of plot_heatmap.ipynb in /home/julienb/Documents/Data/Luisa/Rat101/opto_Fastest/Session--Rat101_20240910/analysis 
---------------------------------------------------------------------------
Exception encountered at "In [6]":
---------------------------------------------------------------------------
StopIteration                             Traceback (most recent call last)
File ~/miniconda3/envs/dev/lib/python3.11/site-packages/xarray/core/concat.py:228, in concat(objs, dim, data_vars, coords, compat, positions, fill_value, join, combine_attrs)
    227 try:
--> 228     first_obj, objs = utils.peek_at(objs)
    229 except StopIteration:

File ~/miniconda3/envs/dev/lib/python3.11/site-packages/xarray/core/utils.py:205, in peek_at(iterable)
    204 gen = iter(iterable)
--> 205 peek = next(gen)
    206 return peek, itertools.chain([peek], gen)

StopIteration: 

During handling of the above exception, another exception occurred:

ValueError                                

None

During execution of plot_heatmap.ipynb in /home/julienb/Documents/Data/Luisa/Rat101/opto_Fastest/Session--Rat101_20240913/analysis 
---------------------------------------------------------------------------
Exception encountered at "In [6]":
---------------------------------------------------------------------------
StopIteration                             Traceback (most recent call last)
File ~/miniconda3/envs/dev/lib/python3.11/site-packages/xarray/core/concat.py:228, in concat(objs, dim, data_vars, coords, compat, positions, fill_value, join, combine_attrs)
    227 try:
--> 228     first_obj, objs = utils.peek_at(objs)
    229 except StopIteration:

File ~/miniconda3/envs/dev/lib/python3.11/site-packages/xarray/core/utils.py:205, in peek_at(iterable)
    204 gen = iter(iterable)
--> 205 peek = next(gen)
    206 return peek, itertools.chain([peek], gen)

StopIteration: 

During handling of the above exception, another exception occurred:

ValueError                                

...

None

During execution of plot_heatmap.ipynb in /home/julienb/Documents/Data/Luisa/Rat246/opto_MT/Session--Rat246_20240730/analysis 
---------------------------------------------------------------------------
Exception encountered at "In [6]":
---------------------------------------------------------------------------
StopIteration                             Traceback (most recent call last)
File ~/miniconda3/envs/dev/lib/python3.11/site-packages/xarray/core/concat.py:228, in concat(objs, dim, data_vars, coords, compat, positions, fill_value, join, combine_attrs)
    227 try:
--> 228     first_obj, objs = utils.peek_at(objs)
    229 except StopIteration:

File ~/miniconda3/envs/dev/lib/python3.11/site-packages/xarray/core/utils.py:205, in peek_at(iterable)
    204 gen = iter(iterable)
--> 205 peek = next(gen)
    206 return peek, itertools.chain([peek], gen)

StopIteration: 

During handling of the above exception, another exception occurred:

ValueError                                Trace

None

,quick_path,script,err_message
0,opto_Fastest/Session--Rat101_20240910,plot_heatmap.ipynb,[]
1,opto_Fastest/Session--Rat101_20240913,plot_heatmap.ipynb,[]
2,opto_MT/Session--Rat101_20240729,plot_heatmap.ipynb,[]
3,opto_CueRT/Session--Rat226_20240906,plot_heatmap.ipynb,[]
4,opto_Fastest/Session--Rat226_20240909,plot_heatmap.ipynb,[]
5,opto_Fastest/Session--Rat226_20240910,plot_heatmap.ipynb,[]
6,opto_Fastest/Session--Rat226_20240911,plot_heatmap.ipynb,[]
7,opto_Fastest/Session--Rat226_20240912,plot_heatmap.ipynb,[]
8,opto_Fastest/Session--Rat226_20240913,plot_heatmap.ipynb,[]
9,opto_MT/Session--Rat226_20240730,plot_heatmap.ipynb,[]


  + Exception Group Traceback (most recent call last):
  |   File "/home/julienb/miniconda3/envs/dev/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_384725/3741885339.py", line 29, in <module>
  |     raise ExceptionGroup("Errors during execution", [e[-1] for e in errors])
  | ExceptionGroup: Errors during execution (36 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/tmp/ipykernel_384725/3741885339.py", line 17, in <module>
    |     papermill.execute_notebook(p/script, p/script.stem/"notebook.ipynb", cwd=p, progress_bar=False,  log_ouput=True)
    |   File "/home/julienb/miniconda3/envs/dev/lib/python3.11/site-packages/papermill/execute.py", line 131, in execute_notebook
    |     raise_for_execution_errors(nb, output_path)
    |   File "/home/julienb/miniconda3/envs/dev/lib/python3.11/site-packages